# Plumbing Inventory Webscraper

In [27]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time
import openpyxl
from itertools import zip_longest

Plumbing Parts and Supplies Tab

In [4]:
url = 'https://www.ferguson.com/category/plumbing-parts-supplies/_/N-zbq4ij?Nrpp=20&_=1650403934744&brand_fr=some&category_fr=some&ds=list&sr=store '

#making sure webpage stays open
options = webdriver.ChromeOptions()
options.add_experimental_option('detach', True)
s = Service(r"C:\Users\jacob\Desktop\py4e\chromedriver.exe")
driver = webdriver.Chrome(service = s)

#open webpage
driver.get(url)

In [5]:
#retrieve the elements I want to loop through
productlist = driver.find_elements(By.CLASS_NAME, 'fg-search-results')

#show that we got the elements we want
for prod in productlist:
    print(prod.text)

PROFLO® 100 Series 1-1/4 x 1-1/2 in. White Plastic Slip Joint Tubular P-Trap with Marvel Adapter
Part #PFPTP100  |  Item #1894413  |  Mfr. Part #FPTP100
1-1/2 in
$2.00 EACH
Add to Cart
172 in Philadelphia, PA - Washington Ave
Check Nearby
PROFLO® 200 Series 3-Piece Soft P-Trap Cover in China White
Part #PF202WH  |  Item #467745  |  Mfr. Part #PF202WH
$66.42 EACH
Add to Cart
7 in Philadelphia, PA - Washington Ave
Check Nearby
PROFLO® 700 Series 16-1/2 in. Slip-Joint Extension Tube in White
Part #PFETP700  |  Item #1894406  |  Mfr. Part #PFETP700
$3.07 EACH
Add to Cart
92 in Philadelphia, PA - Washington Ave
Check Nearby
PROFLO® 100 Series 1-1/2 x 1-1/2 in. Adjustable Tubular Brass P-Trap
Part #PFPTB107  |  Item #1619040  |  Mfr. Part #PFPTB107
$16.58 EACH
Add to Cart
56 in Philadelphia, PA - Washington Ave
Check Nearby
PROFLO® 200 Series 13 in. Slip-Joint Disposer Kit in White
Part #PFEWP200  |  Item #1894426  |  Mfr. Part #PFEWP200
$4.73 EACH
Add to Cart
0 in Philadelphia, PA - Washing

In [6]:
#converting the list of 1 item that is a webdriver object into a list of strings
#will make it easy to extract the title, price, and how many are in stock
strlist = [prod.text for prod in productlist]

strlist
#is a list of len 1 that consists of the string version of the webobject from the extracted website

['PROFLO® 100 Series 1-1/4 x 1-1/2 in. White Plastic Slip Joint Tubular P-Trap with Marvel Adapter\nPart #PFPTP100  |  Item #1894413  |  Mfr. Part #FPTP100\n1-1/2 in\n$2.00 EACH\nAdd to Cart\n172 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 200 Series 3-Piece Soft P-Trap Cover in China White\nPart #PF202WH  |  Item #467745  |  Mfr. Part #PF202WH\n$66.42 EACH\nAdd to Cart\n7 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 700 Series 16-1/2 in. Slip-Joint Extension Tube in White\nPart #PFETP700  |  Item #1894406  |  Mfr. Part #PFETP700\n$3.07 EACH\nAdd to Cart\n92 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 100 Series 1-1/2 x 1-1/2 in. Adjustable Tubular Brass P-Trap\nPart #PFPTB107  |  Item #1619040  |  Mfr. Part #PFPTB107\n$16.58 EACH\nAdd to Cart\n56 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 200 Series 13 in. Slip-Joint Disposer Kit in White\nPart #PFEWP200  |  Item #1894426  |  Mfr. Part #PFEWP200\n$4.73 EACH\nAdd to Cart\n

In [7]:
#breaking list up into individual characters by moving into a new list
brokenlist = []

for item in strlist:
    for character in item:
        brokenlist.append(character)

#putting it back together into a string
joinedstr = ''.join(brokenlist)

joinedstr

'PROFLO® 100 Series 1-1/4 x 1-1/2 in. White Plastic Slip Joint Tubular P-Trap with Marvel Adapter\nPart #PFPTP100  |  Item #1894413  |  Mfr. Part #FPTP100\n1-1/2 in\n$2.00 EACH\nAdd to Cart\n172 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 200 Series 3-Piece Soft P-Trap Cover in China White\nPart #PF202WH  |  Item #467745  |  Mfr. Part #PF202WH\n$66.42 EACH\nAdd to Cart\n7 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 700 Series 16-1/2 in. Slip-Joint Extension Tube in White\nPart #PFETP700  |  Item #1894406  |  Mfr. Part #PFETP700\n$3.07 EACH\nAdd to Cart\n92 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 100 Series 1-1/2 x 1-1/2 in. Adjustable Tubular Brass P-Trap\nPart #PFPTB107  |  Item #1619040  |  Mfr. Part #PFPTB107\n$16.58 EACH\nAdd to Cart\n56 in Philadelphia, PA - Washington Ave\nCheck Nearby\nPROFLO® 200 Series 13 in. Slip-Joint Disposer Kit in White\nPart #PFEWP200  |  Item #1894426  |  Mfr. Part #PFEWP200\n$4.73 EACH\nAdd to Cart\n0

In [8]:
#split string into a list of strings by '\n'
cleanlist = joinedstr.split('\n')

cleanlist

['PROFLO® 100 Series 1-1/4 x 1-1/2 in. White Plastic Slip Joint Tubular P-Trap with Marvel Adapter',
 'Part #PFPTP100  |  Item #1894413  |  Mfr. Part #FPTP100',
 '1-1/2 in',
 '$2.00 EACH',
 'Add to Cart',
 '172 in Philadelphia, PA - Washington Ave',
 'Check Nearby',
 'PROFLO® 200 Series 3-Piece Soft P-Trap Cover in China White',
 'Part #PF202WH  |  Item #467745  |  Mfr. Part #PF202WH',
 '$66.42 EACH',
 'Add to Cart',
 '7 in Philadelphia, PA - Washington Ave',
 'Check Nearby',
 'PROFLO® 700 Series 16-1/2 in. Slip-Joint Extension Tube in White',
 'Part #PFETP700  |  Item #1894406  |  Mfr. Part #PFETP700',
 '$3.07 EACH',
 'Add to Cart',
 '92 in Philadelphia, PA - Washington Ave',
 'Check Nearby',
 'PROFLO® 100 Series 1-1/2 x 1-1/2 in. Adjustable Tubular Brass P-Trap',
 'Part #PFPTB107  |  Item #1619040  |  Mfr. Part #PFPTB107',
 '$16.58 EACH',
 'Add to Cart',
 '56 in Philadelphia, PA - Washington Ave',
 'Check Nearby',
 'PROFLO® 200 Series 13 in. Slip-Joint Disposer Kit in White',
 'Part 

In [9]:
#extracting all itemnames and brands into a new list
itemNames = []
brands = ('PROFLO', 'Jones', 'KOHLER', 'Delta', 'T&S', 'American', 'Moen', 'Sloan', 'Pfister', 'Oatey',
         'Fluidmaster', 'Lincoln', 'Central', 'Hercules', 'Sioux', 'Keeney', 'Lavelle', 'Rectorseal', 
         'Korky', 'Fernco', 'General Pipe Cleaners', 'InSinkErator', 'Jay R. Smith', 'PROSELECT', 
         'Utility Manufacturing', 'Brass Craft', 'Brizo', 'Cariff', 'FNW', 'Flushmate', 'GROHE',
         'Gerber', 'Goss', 'La-Co', 'SFA Saniflo', 'SharkBite', 'William H. Harvey')

for line in cleanlist:
    if line.startswith(brands):
        itemNames.append(line)
        
print(len(itemNames))
itemNames

751


['PROFLO® 100 Series 1-1/4 x 1-1/2 in. White Plastic Slip Joint Tubular P-Trap with Marvel Adapter',
 'PROFLO® 200 Series 3-Piece Soft P-Trap Cover in China White',
 'PROFLO® 700 Series 16-1/2 in. Slip-Joint Extension Tube in White',
 'PROFLO® 100 Series 1-1/2 x 1-1/2 in. Adjustable Tubular Brass P-Trap',
 'PROFLO® 200 Series 13 in. Slip-Joint Disposer Kit in White',
 'PROFLO® 16 in. Flanged Tailpiece in White',
 'PROFLO® PFPTB100 Series 1-1/4 in. Brass P-Trap in Polished Chrome',
 'PROFLO® 200 Series Soft Offset Tailpiece and P-Trap Supply Cover in China White',
 'PROFLO® 100 Series 16 in. Direct Connect End Outlet Waste in White',
 'PROFLO® 400 Series 1-1/4 x 1-1/4 in. Adjustable Tubular Brass P-Trap with Semi-Cast Body with Cleanout',
 'PROFLO® PFPTB400 Series 1-1/2 in. Brass P-Trap in Polished Chrome',
 'PROFLO® 100 Series 1-1/2 in. Brass P-Trap',
 'PROFLO® PFEWP100 Series 16 in. Slip-Joint End Outlet Waste in White',
 'PROFLO® 700 Series 16-1/2 in. 20 ga Slip-Joint Extension Tube 

In [10]:
#extracting all prices and putting them into new list
prices = []

for line in cleanlist:
    if line.startswith('$'):
        prices.append(line[0:5])
        
print(len(prices))
prices

752


['$2.00',
 '$66.4',
 '$3.07',
 '$16.5',
 '$4.73',
 '$2.11',
 '$15.9',
 '$83.2',
 '$5.33',
 '$42.7',
 '$43.3',
 '$32.6',
 '$4.83',
 '$24.7',
 '$2.66',
 '$24.3',
 '$2.31',
 '$3.18',
 '$26.5',
 '$33.5',
 '$6.24',
 '$2.62',
 '$14.1',
 '$1.53',
 '$8.05',
 '$10.6',
 '$85.3',
 '$85.3',
 '$10.4',
 '$26.3',
 '$53.1',
 '$27.4',
 '$104.',
 '$104.',
 '$104.',
 '$104.',
 '$46.9',
 '$51.2',
 '$4.65',
 '$46.9',
 '$66.2',
 '$65.8',
 '$3.75',
 '$84.6',
 '$87.6',
 '$113.',
 '$3.78',
 '$25.6',
 '$2.18',
 '$110.',
 '$53.3',
 '$60.2',
 '$17.8',
 '$79.3',
 '$16.6',
 '$21.9',
 '$11.9',
 '$74.0',
 '$29.8',
 '$17.5',
 '$4.91',
 '$20.0',
 '$33.6',
 '$34.7',
 '$25.0',
 '$34.5',
 '$6.30',
 '$74.0',
 '$32.2',
 '$28.9',
 '$56.9',
 '$94.9',
 '$38.4',
 '$36.8',
 '$47.5',
 '$35.5',
 '$57.6',
 '$53.5',
 '$29.9',
 '$36.9',
 '$21.9',
 '$128.',
 '$15.6',
 '$39.9',
 '$24.4',
 '$13.3',
 '$90.5',
 '$96.5',
 '$41.9',
 '$13.3',
 '$43.8',
 '$35.8',
 '$28.9',
 '$31.7',
 '$91.6',
 '$10.4',
 '$119.',
 '$69.1',
 '$3.99',
 '$25.9',


In [11]:
#extracting all inventory left and putting them into new list
inventoryList = []

for line in cleanlist:
    if line.find('Philadelphia') != -1:
        inventoryList.append(line[0:line.index('i')])

#lens are different b/c website required login to see entire list. Cross-checked price with items to verify
#For my current purpose, inventory remaining are arbitrary so I will randomly fill in the rest
#will fix later if used for actual business application
print(len(inventoryList)) 
inventoryList

717


['172 ',
 '7 ',
 '92 ',
 '56 ',
 '0 ',
 '13 ',
 '35 ',
 '0 ',
 '48 ',
 '0 ',
 '0 ',
 '0 ',
 '7 ',
 '0 ',
 '0 ',
 '0 ',
 '0 ',
 '125 ',
 '0 ',
 '0 ',
 '7 ',
 '5 ',
 '3 ',
 '8 ',
 '11 ',
 '9 ',
 '1 ',
 '1 ',
 '5 ',
 '6 ',
 '7 ',
 '16 ',
 '10 ',
 '1 ',
 '1 ',
 '12 ',
 '8 ',
 '87 ',
 '18 ',
 '9 ',
 '20 ',
 '73 ',
 '3 ',
 '7 ',
 '2 ',
 '51 ',
 '9 ',
 '91 ',
 '3 ',
 '9 ',
 '46 ',
 '5 ',
 '1 ',
 '20 ',
 '3 ',
 '3 ',
 '8 ',
 '25 ',
 '20 ',
 '2 ',
 '2 ',
 '8 ',
 '3 ',
 '6 ',
 '7 ',
 '3 ',
 '6 ',
 '2 ',
 '2 ',
 '3 ',
 '2 ',
 '2 ',
 '1 ',
 '1 ',
 '1 ',
 '6 ',
 '2 ',
 '11 ',
 '4 ',
 '2 ',
 '2 ',
 '2 ',
 '2 ',
 '2 ',
 '4 ',
 '4 ',
 '7 ',
 '2 ',
 '6 ',
 '4 ',
 '2 ',
 '5 ',
 '2 ',
 '6 ',
 '13 ',
 '2 ',
 '12 ',
 '2 ',
 '301 ',
 '30 ',
 '1 ',
 '5 ',
 '4 ',
 '3 ',
 '3 ',
 '2 ',
 '3 ',
 '6 ',
 '1 ',
 '2 ',
 '4 ',
 '5 ',
 '3 ',
 '3 ',
 '4 ',
 '3 ',
 '133 ',
 '1 ',
 '7 ',
 '1 ',
 '5 ',
 '2 ',
 '2 ',
 '2 ',
 '3 ',
 '4 ',
 '4 ',
 '1 ',
 '1 ',
 '2 ',
 '2 ',
 '2 ',
 '1 ',
 '16 ',
 '11 ',
 '3 ',
 '36 ',
 '20 ',

In [12]:
#extracting all partID and itemID, will split up later in excel
IDs = []

for line in cleanlist:
    if line.startswith('Part #'):
        IDs.append(line)

#Will clean later in excel
print(len(IDs))
IDs

754


['Part #PFPTP100  |  Item #1894413  |  Mfr. Part #FPTP100',
 'Part #PF202WH  |  Item #467745  |  Mfr. Part #PF202WH',
 'Part #PFETP700  |  Item #1894406  |  Mfr. Part #PFETP700',
 'Part #PFPTB107  |  Item #1619040  |  Mfr. Part #PFPTB107',
 'Part #PFEWP200  |  Item #1894426  |  Mfr. Part #PFEWP200',
 'Part #PFTPP200  |  Item #1894437  |  Mfr. Part #PFTPP200',
 'Part #PFPTB100  |  Item #1619039  |  Mfr. Part #PFPTB100',
 'Part #PF203WH  |  Item #467746  |  Mfr. Part #PF203WH',
 'Part #PFEWP101  |  Item #1894427  |  Mfr. Part #PFEWP101',
 'Part #PFPTB400  |  Item #1375944  |  Mfr. Part #PFPTB400',
 'Part #PFPTB403  |  Item #1375946  |  Mfr. Part #PFPTB403',
 'Part #PFPTB103  |  Item #1619038  |  Mfr. Part #PFPTB103',
 'Part #PFEWP100  |  Item #2193312  |  Mfr. Part #12516W',
 'Part #PFETB702  |  Item #4055894  |  Mfr. Part #PFETB702',
 'Part #PFTPP401  |  Item #1894430  |  Mfr. Part #FTPP401',
 'Part #PFPTB110  |  Item #2026478  |  Mfr. Part #PFPTB110',
 'Part #PFETP600  |  Item #1894439

In [31]:
#creating a pandas dataframe to easily import to excel for cleaning and analysis
df = pd.DataFrame(list(zip_longest(itemNames, prices, inventoryList, IDs, fillvalue = None)),
                  columns = ['ItemName', 'Price', '#left', 'IDs'])

df

,ItemName,Price,#left,IDs
0,PROFLO® 100 Series 1-1/4 x 1-1/2 in. White Pla...,$2.00,172,Part #PFPTP100 | Item #1894413 | Mfr. Part...
1,PROFLO® 200 Series 3-Piece Soft P-Trap Cover i...,$66.4,7,Part #PF202WH | Item #467745 | Mfr. Part #...
2,PROFLO® 700 Series 16-1/2 in. Slip-Joint Exten...,$3.07,92,Part #PFETP700 | Item #1894406 | Mfr. Part...
3,PROFLO® 100 Series 1-1/2 x 1-1/2 in. Adjustabl...,$16.5,56,Part #PFPTB107 | Item #1619040 | Mfr. Part...
4,PROFLO® 200 Series 13 in. Slip-Joint Disposer ...,$4.73,0,Part #PFEWP200 | Item #1894426 | Mfr. Part...
...,...,...,...,...
749,Oatey® 8 oz Plastic Grey Pipe Cement,$11.6,None,Part #JC25464 | Item #9212169 | Mfr. Part ...
750,American Standard 3 in. Silicone Flapper in White,$8.53,None,Part #H25515 | Item #22766 | Mfr. Part #25515
751,None,$26.6,None,Part #O30834 | Item #16706 | Mfr. Part #30834
752,None,None,None,Part #O31094 | Item #928700 | Mfr. Part #3...


In [35]:
#writing to excel, final step for python!
df.to_excel(r'C:\Users\jacob\Desktop\pandas-to-excel1.xlsx')
#analyze mcmastercarr